# Code to measure the offset of each MIRI galaxy to the centre of the frame

To do this, we will read in the catalogue of galaxies to obtain their respective IDs, RAs and Decs. Then we will fit a centroid to all our MIRI images and determine their coordinates. From this we can calculate the spherical offsets. Lastly we will plot the results and see what happens.

The code is separated into several functions that are being called by one main function for better readability and easier debugging.

In [ ]:
from astropy.io import fits
from astropy import table
import numpy as np
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
from astropy.nddata import Cutout2D
from photutils import centroids
import scipy
import os
import glob
import pandas as pd

Define the necessary functions:

In [ ]:

def read_catalogue(catalogue, survey):
    """Reads the catalogue file and adds columns for astrometric offsets."""
    cat = table.Table.read(catalogue, 1)
    cat[f'{survey}_dra'] = 0.0 * u.arcsec
    cat[f'{survey}_ddec'] = 0.0 * u.arcsec
    return cat

def load_cutout(file_pattern, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_pattern) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_pattern}")
        return None, None

def compute_centroid(cutout, smooth_sigma, good_frac_cutout, smooth_miri):
    """Computes the centroid using a smoothed version of the image."""
    if cutout is None:
        return None

    # Decide whether to smooth MIRI or not
    if smooth_miri == True:
        smoothed_data = scipy.ndimage.gaussian_filter(cutout.data, smooth_sigma)
    else: 
        smoothed_data = cutout.data
    
    search_boxsize = int(np.floor(good_frac_cutout * cutout.shape[0]) // 2 * 2 + 1)

    centroid_pix = centroids.centroid_quadratic(
        smoothed_data,
        xpeak=cutout.shape[0] // 2,
        ypeak=cutout.shape[1] // 2,
        search_boxsize=search_boxsize,
        fit_boxsize=5
    )

    return cutout.wcs.pixel_to_world(centroid_pix[0], centroid_pix[1]) if not np.isnan(centroid_pix).any() else None

def save_alignment_figure(g, cutout_nircam, cutout_miri, centroid_nircam, centroid_miri, output_dir, survey, filter):
    """Saves a figure showing centroid alignment."""
    fig, axs = plt.subplots(1, 2, figsize=[10, 5])

    axs[0].imshow(scipy.ndimage.gaussian_filter(cutout_nircam.data, 1.0), origin='lower')
    axs[0].plot(*cutout_nircam.wcs.world_to_pixel(centroid_nircam), 'x', color='red')
    axs[0].set(title=f"NIRCam F444W Reference {g['id']}")

    axs[1].imshow(scipy.ndimage.gaussian_filter(cutout_miri.data, 1.0), origin='lower')
    axs[1].plot(*cutout_miri.wcs.world_to_pixel(centroid_miri), 'o', color='orange')
    axs[1].set(title=f"MIRI {filter} Cutout {g['id']}")
    
    # show expected position of the centroid
    expected_position_pix = cutout_miri.wcs.world_to_pixel(centroid_nircam)
    axs[1].plot(expected_position_pix[0], expected_position_pix[1], 'x', color='red')

    output_path = os.path.join(output_dir, f"{g['id']}_{filter}_offset_{survey}.pdf")
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(output_path)
    plt.close()

    print(f"Saved figure: {output_path}")


def compute_offset(catalogue, cutout_folder, survey, filter, obs="", smooth_miri=True):
    """Computes the astrometric offset between NIRCam and MIRI for each galaxy."""
    cat = read_catalogue(catalogue, survey)

    for i, g in enumerate(cat):
        print(f"Processing galaxy {g['id']}...")

        ref_position = SkyCoord(ra=g['ra'], dec=g['dec'], unit=u.deg)
        cutout_size = (2.5 * u.arcsec, 2.5 * u.arcsec)
        smooth_sigma, good_frac_cutout = 1.0, 0.7

        # Load MIRI cutout
        cutout_miri_path = os.path.join(cutout_folder, f"{g['id']}_{filter}_cutout_{survey}{obs}_rot.fits")
        miri_data, miri_wcs = load_cutout(cutout_miri_path)
        if miri_data is None:
            continue
        cutout_miri = Cutout2D(miri_data, ref_position, cutout_size, wcs=miri_wcs)

        # Load NIRCam cutout
        nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{g['id']}_F444W_cutout.fits"
        nircam_data, nircam_wcs = load_cutout(nircam_path)
        if nircam_data is None:
            continue
        cutout_nircam = Cutout2D(nircam_data, ref_position, cutout_size, wcs=nircam_wcs)

        # Compute centroids
        centroid_nircam = compute_centroid(cutout_nircam, smooth_sigma, good_frac_cutout, smooth_miri)
        centroid_miri = compute_centroid(cutout_miri, smooth_sigma, good_frac_cutout, smooth_miri)

        if centroid_nircam is None or centroid_miri is None:
            print("Centroid not found for one or both cutouts. Skipping.")
            continue

        # Save alignment figure
        output_dir = f"/home/bpc/University/master/Red_Cardinal/offsets/{survey}{obs}/"
        save_alignment_figure(g, cutout_nircam, cutout_miri, centroid_nircam, centroid_miri, output_dir, survey, filter)

        # Compute offsets
        dra, ddec = centroid_nircam.spherical_offsets_to(centroid_miri)
        cat[f'{survey}_dra'][i] = dra.to(u.arcsec).value
        cat[f'{survey}_ddec'][i] = ddec.to(u.arcsec).value

        print(f"Offset: ΔRA = {dra.to(u.arcsec)}, ΔDec = {ddec.to(u.arcsec)}")

    return cat


Load the necessary data files

In [ ]:
catalogue = '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

# Specify the path to the cutouts directory
my_cutouts = '/home/bpc/University/master/Red_Cardinal/cutouts/'
co_rotated = '/home/bpc/University/master/Red_Cardinal/cutouts_rotated/'

# Load the PRIMER cutouts
primer003_rot = glob.glob(os.path.join(co_rotated, "*primer003_rot.fits"))
print(f"Found {len(primer003_rot)} FITS files for the PRIMER survey.")

# Load the PRIMER cutouts
primer004_rot = glob.glob(os.path.join(co_rotated, "*primer004_rot.fits"))
print(f"Found {len(primer004_rot)} FITS files for the PRIMER survey.")

# Load the COSMOS-Web cutouts
cweb_rot = glob.glob(os.path.join(co_rotated, "*cweb_rot.fits"))
print(f"Found {len(cweb_rot)} FITS files for the COSMOS-Web survey.")



In [ ]:
compute_offset(catalogue, co_rotated, 'primer', 'F770W', '003')
compute_offset(catalogue, co_rotated, 'primer', 'F770W', '004')
compute_offset(catalogue, co_rotated, survey='cweb', filter='F770W', obs="")